In [ ]:
import pandas as pd
import urllib.request as req
from bs4 import BeautifulSoup as bs
from multiprocessing.pool import ThreadPool
import numpy as np

In [ ]:
df = pd.read_csv('./db.csv', index_col=0)
df

In [ ]:
def isGood(url: str) -> bool:
    if url.startswith('https://stackoverflow.com'):
        if 'jobs' not in url:
            if 'developer' not in url:
                if 'directory' not in url:
                    return True
    return False

def find_is_closed(url: str):
    if not isGood(url):
        return False
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    request=req.Request(url,None,headers) #The assembled request
    try:
        response = req.urlopen(request, timeout=2)
    except Exception as e:
        # print(f'{url=} encountered an error {e}')
        return False
    data = response.read()
    soup = bs(data, 'html.parser')
    aside = soup.findAll('aside')
    if len(aside)<1:
        return False
    soup_aside = bs(str(aside[0]))
    b = soup_aside.find('b')
    if not b:
        return False
    b = b.text.lower()
    return ('close' in b or 'answer' in b)

In [ ]:
with ThreadPool(12) as pool:
    a = pool.map(find_is_closed, df['url'])

In [ ]:
df['cat'] = list(map(int,a))

In [ ]:
# df.to_csv('db_cat.csv')
